In [1]:
import tempfile

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from IPython.display import clear_output

In [2]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation = 'relu', input_shape = (4,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3)
])

In [3]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              optimizer = 'adam')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                80        
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 51        
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Create input_fn

def input_fn():
  split = tfds.Split.TRAIN
  dataset = tfds.load('iris', split = split, as_supervised = True)
  dataset = dataset.map(lambda features, labels: ({'dense_input':features}, labels))
  dataset = dataset.batch(32).repeat()
  return dataset

In [5]:
# Test input_fn

for features_batch, labels_batch in input_fn().take(1):
  print(features_batch)
  print(labels_batch)
    
clear_output()

In [6]:
# Create estimator from keras

model_dir = tempfile.mkdtemp()
keras_estimator = tf.keras.estimator.model_to_estimator(
    keras_model = model, model_dir = model_dir)

clear_output()

In [7]:
keras_estimator.train(input_fn = input_fn, steps = 500)
eval_result = keras_estimator.evaluate(input_fn = input_fn, steps = 10)
clear_output()
print('Eval result: {}'.format(eval_result))

Eval result: {'loss': 0.37494054, 'global_step': 500}
